# Aries Basic Controller - Issuer Example

This notebook walks through how to issue a credential across a previously established connection.

**Note: You should first complete the DIDExchange example notebooks to establish a connection between Alice and Bob**

* [Alice](http://localhost:8888/notebooks/did-exchange-inviter.ipynb)
* [Bob](http://localhost:8889/notebooks/did-exchange-invitee.ipynb)


In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)
    

## Register Listeners

Make sure you have terminated the agent controller being used in the did-exchange example or it will throw an error.

In [4]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([], defaults=True)

## Check the agent has an active connection

If no connections returned, or the connection not active then this tutorial will not work. You will need to revisit the did-exchange notebooks for Alice and Bob.

In [5]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    


Results :  [{'updated_at': '2020-07-03 11:44:28.525278Z', 'invitation_mode': 'once', 'routing_state': 'none', 'invitation_key': '9ywxqPKtayXpyiQ7K8YedLwJq5osGpJ6HUELvdCv4hdr', 'their_did': 'WMt1wVYjkRNGsWt2jaXvLS', 'state': 'active', 'accept': 'manual', 'initiator': 'self', 'my_did': 'G4c8DyQsdbBWnQZfyPtXct', 'their_label': 'Bob', 'created_at': '2020-07-03 11:43:56.161836Z', 'connection_id': '5d465532-247d-47ce-a56f-c4ef65e7a5b9'}]
Connection : {'updated_at': '2020-07-03 11:44:28.525278Z', 'invitation_mode': 'once', 'routing_state': 'none', 'invitation_key': '9ywxqPKtayXpyiQ7K8YedLwJq5osGpJ6HUELvdCv4hdr', 'their_did': 'WMt1wVYjkRNGsWt2jaXvLS', 'state': 'active', 'accept': 'manual', 'initiator': 'self', 'my_did': 'G4c8DyQsdbBWnQZfyPtXct', 'their_label': 'Bob', 'created_at': '2020-07-03 11:43:56.161836Z', 'connection_id': '5d465532-247d-47ce-a56f-c4ef65e7a5b9'}
Active Connection ID :  5d465532-247d-47ce-a56f-c4ef65e7a5b9


## Write a Schema to the Ledger

For more details see the [schema-api notebook](http://localhost:8888/notebooks/schema_api.ipynb)

In [6]:
# Define you schema name - must be unique on the ledger
schema_name = "open_mined_contributor"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["name", "skill"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)


PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1


## Write a Credential Definition to the Ledger

More details in the [definitions notebook](http://localhost:8888/notebooks/definitions_api.ipynb)

In [7]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default


**Note: You should be able to see both schema and definitions transactions on the local network [here](http://localhost:9000)**

## Populate the Attribues to Issue to Bob

The schema defines two attributes: name and skill. Both must be populated with attribute values that Alice wishes to issue in a credential to Bob. To do this a list of objects must be created, each object containing the name of the attribute and it's value at a minimum. You can set the values to anything you wish.

TODO: Some additional fields such as mime-type can be defined.

In [8]:
credential_attributes = [
    {"name": "name", "value": "Bob", "mime-type": "text/plain"},
    {"name": "skill", "value": "researcher", "mime-type": "text/plain"}
]
print(credential_attributes)

[{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]


## Send Credential

This sends a credential to a holder (Bob), and automates the rest of the protocol. There are other ways to issue a credential that require more stages.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to True. Not entirely sure about this one, believe its for logging.

In [9]:
response = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes)
print(response)

Schema issuer PQRXDxdGqQGSZ8z69p4xZP
Cred issuer PQRXDxdGqQGSZ8z69p4xZP


ClientResponseError: 500, message='Internal Server Error', url=URL('http://alice-agent:8021/issue-credential/send')

Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/aries_basic_controller/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/aries_basic_controller/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/aries_basic_controller/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit

In [15]:
print(connection_id)
response = await agent_controller.connections.get_connection(connection_id)
print(response["state"])

436c3c3f-83a3-4834-adb3-d8f23745fcb4
active


In [22]:
schema_issuer = schema_id.split(":")[0]
print(schema_issuer)

PQRXDxdGqQGSZ8z69p4xZP


In [23]:
cred_def_issuer = cred_def_id.split(":")[0]
print(cred_def_issuer)

PQRXDxdGqQGSZ8z69p4xZP
